In [4]:
from datasets import load_dataset

dataset = load_dataset("NbAiLab/norwegian_parliament")


Using custom data configuration default
Reusing dataset norwegian_parliament (/cluster/home/jorgevh/.cache/huggingface/datasets/NbAiLab___norwegian_parliament/default/1.0.0/0ef7cafbd6791012727b235682443606a30da1177ae233308cbd4e6801c38fe8)
100%|██████████| 3/3 [00:00<00:00,  3.07it/s]


In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import transformers
transformers.utils.logging.set_verbosity_debug()

model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like meta-llama/Llama-2-7b-hf is not the path to a directory containing a {configuration_file} file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)

train_dataset = dataset["train"].map(tokenize, batched=True)
eval_dataset = dataset["validation"].map(tokenize, batched=True)
test_dataset = dataset["test"].map(tokenize, batched=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()

In [ ]:
trainer.evaluate()